In [1394]:
import pandas as pd
import numpy as np
import os 

In [1395]:
cwd = os.getcwd()

df = pd.read_excel(cwd+"/B2B Data Provider-sample1.xlsx", index_col=0)

#replace all Nan values with -
df = df.replace(np.nan, "-")

In [1396]:
print(df.columns)

Index(['Starting Price - Monthly', 'Starting Price - Annually',
       'Pricing Range', 'Pricing comments', 'Additional Pricing Info',
       'Website', 'Contract', 'Solutions', 'Unlimited Data Plan',
       'CRM Integrations', 'SEPI', 'SalesTeamType', 'Free Trial',
       'ContactEnrichment', 'IntentData', 'Other', 'location'],
      dtype='object')


In [1397]:
#removing unnecessary columns
df_soft = df.drop(['Additional Pricing Info','Website','Unlimited Data Plan', 'Pricing Range', 'Pricing comments', 'SalesTeamType','Free Trial','Other'], axis=1)

#drop the pricing from data and this can be used to create a feature matrix
df_pro = df_soft.drop(['Starting Price - Monthly', 'Starting Price - Annually'], axis=1)
df_pro

,Contract,Solutions,CRM Integrations,SEPI,ContactEnrichment,IntentData,location
Company,,,,,,,
ZoomInfo,Annual,"email, workphone, mobile","Microsoft Dynamics,HubSpot,Salesforce,Pipedriv...","Outreach,Salesloft",Yes,Yes,"Domestic,International"
Cognism,Annual,"email, workphone, mobile","Salesforce,HubSpot,Pipedrive,Microsoft Dynamics",Salesloft,Yes,Yes,International
Uplead,Annual or Monthly,"email, workphone, mobile","Salesforce,HubSpot,Insightly,Pipedrive,Zoho,Mi...","Outreach,Salesloft,Reply,Woodpecker,Lemlist,Close",Yes,Yes,"Domestic,International"
Lead411,Annual or Monthly,"email, workphone, mobile","Salesforce,HubSpot,Pipedrive,Microsoft Dynamic...",-,Yes,Yes,"Domestic,International"
Apollo,Annual or Monthly,"email, workphone, mobile","Salesforce,HubSpot","Salesloft,Outreach",Yes,No,"Domestic,International"
Seamless,Annual,"email, workphone, mobile","Salesforce,HubSpot,Microsoft Dynamics,Keap,Zoh...","Salesloft,Outreach",Yes,No,"Domestic,International"
RocketReach,Annual or Monthly,"email, workphone, mobile","Salesforce,HubSpot","Salesloft,Outreach",No,No,"Domestic,International"
Lusha,Annual or Monthly,"email, workphone","Salesforce,HubSpot","Salesloft,Outreach",Yes,No,"Domestic,International"
Crunchbase,Annual or Monthly,email,Salesforce,-,No,No,"Domestic,International"


In [1398]:
#convert all the contract to list values
df_pro['Contract'] = df_pro['Contract'].str.replace(" or ", ",").str.split(',')
#convert all solutions values to list 
df_pro['Solutions'] = df_pro['Solutions'].str.split(",")
#convert all the CRM values to list
df_pro['CRM Integrations'] = df_pro['CRM Integrations'].str.split(",")
#convert all SEPI values to list
df_pro['SEPI'] = df_pro['SEPI'].str.split(",")
#convert all location values to list
df_pro['location'] = df_pro['location'].str.split(",")

In [1399]:
#set the contract first
for index,row in df_pro.iterrows():
    for contract in row['Contract']:
        df_pro.at[index,contract] =1
    

In [1400]:
#set the solutions 
for index,row in df_pro.iterrows():
    for sol in row['Solutions']:
        df_pro.at[index,sol] =1
    

In [1401]:
#set the CRM
for index,row in df_pro.iterrows():
    for crm in row['CRM Integrations']:
        df_pro.at[index,crm] = 1

In [1402]:
#set the Sales engagement matrix
for index,row in df_pro.iterrows():
    for sepi in row['SEPI']:
        df_pro.at[index,sepi] = 1

In [1403]:
#set the location matric
for index,row in df_pro.iterrows():
    for loc in row['location']:
        df_pro.at[index,loc] = 1

In [1404]:
#replace the Yes and no values to 1 and 0 respectively for ContactEnrichment and IntentData columns
df_pro = df_pro.replace('Yes', 1.0)
df_pro = df_pro.replace('No', 0)


In [1405]:
#now drop all the initial columns that is not a matrix
df_pro =df_pro.drop(['Contract','Solutions','CRM Integrations','SEPI','location', '-'], axis=1)

In [1406]:
#lastly replace all the nan values with 0
df_pro= df_pro.replace(np.nan, 0)

In [1407]:
#check the shape
df_pro.shape

(10, 32)

In [1408]:
#treated dataframe 
df_pro

,ContactEnrichment,IntentData,Annual,Monthly,email,workphone,mobile,Microsoft Dynamics,HubSpot,Salesforce,...,Freshsales,Outreach,Salesloft,Reply,Woodpecker,Lemlist,Close,Freshworks,Domestic,International
Company,,,,,,,,,,,,,,,,,,,,,
ZoomInfo,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
Cognism,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Uplead,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
Lead411,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
Apollo,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
Seamless,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
RocketReach,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
Lusha,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
Crunchbase,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


**Now the idea is to cross multiply the values user values with and find the weighted values**

## USER data

In [1409]:
df_user = pd.read_csv("user_data.csv", index_col=0)
df_user

,data,contract,duration,price,Location,user,tools,features,crm
0,['Work Phone'],[],['Annual'],20000,['All'],8,['All'],"['ContractEnrichment', 'IntentData']",['All']


In [1410]:
df_user['data'] = df_user['data'].item().replace("'", "").strip("[]").lower()
df_user['data'] = df_user['data'].str.split(",")

In [1411]:
df_user['contract'] = df_user['contract'].item().replace("'", "").strip("[]")
df_user['contract'] = df_user['contract'].str.split(",")

In [1412]:
df_user['duration'] = df_user['duration'].item().replace("'", "").strip("[]")

In [1413]:
df_user['price'] = df_user['price'].item()

In [1414]:
if df_user['Location'].item().replace("'", "").strip("[]") == "All":
    df_user['Location'] = [['International', 'Domestic']]


In [1415]:
df_user['features'] = df_user['features'].item().replace("'", "").strip("[]")
df_user['features'] = df_user['features'].str.split(",")

In [1416]:
if df_user['tools'].item().replace("'", "").strip("[]").replace(" ","") == "All":
    df_user['tools']= [['Outreach', 'Salesloft','Reply','Woodpecker','Lemlist','Close','HubSpot','Salesforce','Pipedrive','Nutshell','Zoho','Freshworks']]
else:
    df_user['tools'] = df_user['tools'].item().replace("'", "").strip("[]")
    df_user['tools'] = df_user['tools'].str.split(",")

In [1417]:
if df_user['crm'].item().replace("'", "").strip("[]").replace(" ","") == "All":
    df_user['crm' ]= [['Microsoft Dynamics','HubSpot','Salesforce','Pipedrive','SugarCRM','Zoho','Insightly','Copper','Nimble','Infusionsoft','Capsule','Nutshell','Base','Vtiger','Keap','Freshsales']]
else:
    df_user['crm'] = df_user['crm'].item().replace("'", "").strip("[]")
    df_user['crm'] = df_user['crm'].str.split(",")

In [1418]:
#we will remove the price for now 
df_user_matrix = df_user.drop(['price','user'], axis=1)


In [1419]:
df_user_matrix

,data,contract,duration,Location,tools,features,crm
0,[work phone],[],Annual,"[International, Domestic]","[Outreach, Salesloft, Reply, Woodpecker, Lemli...","[ContractEnrichment, IntentData]","[Microsoft Dynamics, HubSpot, Salesforce, Pipe..."


In [1420]:
#calculate the total number of tools for crm and sales engagement. We will assign a weighted value to each tool
total_tools = len(df_user_matrix['tools'].item()) + len(df_user_matrix['crm'].item())


In [1421]:
#set the data first
for index,row in df_user_matrix.iterrows():
    for data in row['data']:
        df_user_matrix.at[index,data] = 1

df_user_matrix

,data,contract,duration,Location,tools,features,crm,work phone
0,[work phone],[],Annual,"[International, Domestic]","[Outreach, Salesloft, Reply, Woodpecker, Lemli...","[ContractEnrichment, IntentData]","[Microsoft Dynamics, HubSpot, Salesforce, Pipe...",1.0


In [1422]:
#set the contract first
for index,row in df_user_matrix.iterrows():
    for contract in row['contract']:
        df_user_matrix.at[index,contract] =1
        
        
df_user_matrix

,data,contract,duration,Location,tools,features,crm,work phone,
0,[work phone],[],Annual,"[International, Domestic]","[Outreach, Salesloft, Reply, Woodpecker, Lemli...","[ContractEnrichment, IntentData]","[Microsoft Dynamics, HubSpot, Salesforce, Pipe...",1.0,1.0


In [1423]:
#set the location first
for index,row in df_user_matrix.iterrows():
    for loc in row['Location']:
        df_user_matrix.at[index,loc] =1
        
df_user_matrix

,data,contract,duration,Location,tools,features,crm,work phone,,International,Domestic
0,[work phone],[],Annual,"[International, Domestic]","[Outreach, Salesloft, Reply, Woodpecker, Lemli...","[ContractEnrichment, IntentData]","[Microsoft Dynamics, HubSpot, Salesforce, Pipe...",1.0,1.0,1.0,1.0


In [1424]:
#set the tools first
for index,row in df_user_matrix.iterrows():
    for tool in row['tools']:
        df_user_matrix.at[index,tool] =1/total_tools
        
        
df_user_matrix

,data,contract,duration,Location,tools,features,crm,work phone,,International,...,Reply,Woodpecker,Lemlist,Close,HubSpot,Salesforce,Pipedrive,Nutshell,Zoho,Freshworks
0,[work phone],[],Annual,"[International, Domestic]","[Outreach, Salesloft, Reply, Woodpecker, Lemli...","[ContractEnrichment, IntentData]","[Microsoft Dynamics, HubSpot, Salesforce, Pipe...",1.0,1.0,1.0,...,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714


In [1425]:
#set the features first
for index,row in df_user_matrix.iterrows():
    for feat in row['features']:
        df_user_matrix.at[index,feat] =1
        print(feat)
        
        
df_user_matrix

ContractEnrichment
 IntentData


,data,contract,duration,Location,tools,features,crm,work phone,,International,...,Lemlist,Close,HubSpot,Salesforce,Pipedrive,Nutshell,Zoho,Freshworks,ContractEnrichment,IntentData
0,[work phone],[],Annual,"[International, Domestic]","[Outreach, Salesloft, Reply, Woodpecker, Lemli...","[ContractEnrichment, IntentData]","[Microsoft Dynamics, HubSpot, Salesforce, Pipe...",1.0,1.0,1.0,...,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,1.0,1.0


In [1426]:
#set the crm first
for index,row in df_user_matrix.iterrows():
    for crm in row['crm']:
        df_user_matrix.at[index,crm] =1/total_tools
        
df_user_matrix

,data,contract,duration,Location,tools,features,crm,work phone,,International,...,SugarCRM,Insightly,Copper,Nimble,Infusionsoft,Capsule,Base,Vtiger,Keap,Freshsales
0,[work phone],[],Annual,"[International, Domestic]","[Outreach, Salesloft, Reply, Woodpecker, Lemli...","[ContractEnrichment, IntentData]","[Microsoft Dynamics, HubSpot, Salesforce, Pipe...",1.0,1.0,1.0,...,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714


In [1427]:
#rename column Work phone
df_user_matrix = df_user_matrix.rename(columns={' work phone':'workphone'})


In [1428]:
df_user_matrix = df_user_matrix.iloc[:,7:]


In [1429]:
df_user_matrix

,work phone,,International,Domestic,Outreach,Salesloft,Reply,Woodpecker,Lemlist,Close,...,SugarCRM,Insightly,Copper,Nimble,Infusionsoft,Capsule,Base,Vtiger,Keap,Freshsales
0,1.0,1.0,1.0,1.0,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,...,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714


In [1430]:
for i,item in enumerate(df_user_matrix):
    item = item.lstrip(" ")
    if item in df_pro.columns:
        df_pro[item] = df_pro[item]+1
        
df_pro

,ContactEnrichment,IntentData,Annual,Monthly,email,workphone,mobile,Microsoft Dynamics,HubSpot,Salesforce,...,Freshsales,Outreach,Salesloft,Reply,Woodpecker,Lemlist,Close,Freshworks,Domestic,International
Company,,,,,,,,,,,,,,,,,,,,,
ZoomInfo,1.0,2.0,1.0,0.0,1.0,1.0,1.0,2.0,2.0,2.0,...,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0
Cognism,1.0,2.0,1.0,0.0,1.0,1.0,1.0,2.0,2.0,2.0,...,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
Uplead,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,...,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0
Lead411,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0
Apollo,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,...,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0
Seamless,1.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0,2.0,2.0,...,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0
RocketReach,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,...,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0
Lusha,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0,2.0,...,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0
Crunchbase,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0


In [1431]:
df_pro.sum(axis=1)

Company
ZoomInfo       42.0
Cognism        38.0
Uplead         49.0
Lead411        46.0
Apollo         38.0
Seamless       44.0
RocketReach    37.0
Lusha          37.0
Crunchbase     32.0
Snov.io        36.0
dtype: float64

In [1432]:
df_soft['recommend']= df_pro.sum(axis=1)
#add back the pricing range column 
df_soft['Pricing Range'] = df['Pricing Range']


In [1433]:
if df_user['duration'].item() == "Monthly":
    result = df_soft[df_soft['Starting Price - Monthly'] <= df_user['price'].item()].sort_values(by='recommend', ascending=False)
elif df_user['duration'].item() == "Annual":
    result = df_soft[df_soft['Starting Price - Annually'] <= df_user['price'].item()].sort_values(by='recommend', ascending=False)
else:
    result = df_soft.sort_values(by='recommend', ascending=False)

In [1434]:
result

,Starting Price - Monthly,Starting Price - Annually,Contract,Solutions,CRM Integrations,SEPI,ContactEnrichment,IntentData,location,recommend,Pricing Range
Company,,,,,,,,,,,
Uplead,99,888,Annual or Monthly,"email, workphone, mobile","Salesforce,HubSpot,Insightly,Pipedrive,Zoho,Mi...","Outreach,Salesloft,Reply,Woodpecker,Lemlist,Close",Yes,Yes,"Domestic,International",49.0,"$888 annual, $99 per month, month to month - ..."
Lead411,99,900,Annual or Monthly,"email, workphone, mobile","Salesforce,HubSpot,Pipedrive,Microsoft Dynamic...",-,Yes,Yes,"Domestic,International",46.0,"$99 per month, month to month-$1800+ per year ..."
Seamless,0,1764,Annual,"email, workphone, mobile","Salesforce,HubSpot,Microsoft Dynamics,Keap,Zoh...","Salesloft,Outreach",Yes,No,"Domestic,International",44.0,$1764 per year to $9528 per year+
ZoomInfo,0,11995,Annual,"email, workphone, mobile","Microsoft Dynamics,HubSpot,Salesforce,Pipedriv...","Outreach,Salesloft",Yes,Yes,"Domestic,International",42.0,"Single user Pro Lite: $11,995 – 5,000 credits ..."
Cognism,0,16500,Annual,"email, workphone, mobile","Salesforce,HubSpot,Pipedrive,Microsoft Dynamics",Salesloft,Yes,Yes,International,38.0,"16,500-27,500+"
Apollo,49,0,Annual or Monthly,"email, workphone, mobile","Salesforce,HubSpot","Salesloft,Outreach",Yes,No,"Domestic,International",38.0,"$49 per month, month to month - $6000 per year+"
RocketReach,80,468,Annual or Monthly,"email, workphone, mobile","Salesforce,HubSpot","Salesloft,Outreach",No,No,"Domestic,International",37.0,$80 per month to $6000 per year+
Lusha,39,348,Annual or Monthly,"email, workphone","Salesforce,HubSpot","Salesloft,Outreach",Yes,No,"Domestic,International",37.0,"$39 per monmth to $1,140 per year+"
Snov.io,39,396,Annual,email,-,"HubSpot,Salesforce,Pipedrive,Nutshell,Zoho,Fre...",No,No,"Domestic,International",36.0,"$39 per month to & 7,380"


In [1435]:
[Email, Work Phone, Mobile]


SyntaxError: invalid syntax (654115703.py, line 1)